<h3>This notebook is designed to demonstrate how to connect to snowflake database using Spark and read the data from table into Spark dataframe</h3>

<h4>Note:- If you are using ICP4D and need to whitelist the ip addresses at snowflake, you can get them using Hamburger menu -> Administration -> Cloud Integrations -> Firewall Configuration </h4>

In [12]:
#Get spark version and scala version as we need to download the driver specific to this versions
spark.version

'3.0.1'

In [13]:
#Get scala version
!echo $SPARK_SCALA_VERSION

2.12


In [14]:
#Reading the docs https://docs.snowflake.com/en/user-guide/spark-connector-install.html#step-4-configure-the-local-spark-cluster-or-amazon-emr-hosted-spark-environment
#indicates that you need two drivers.

# For Snowflake JDBC we need to use following jar hosted in maven repo https://mvnrepository.com/artifact/net.snowflake/snowflake-jdbc/3.12.17

# For Snowflake Spark connector, we need to use following jar hosted in maven repo https://mvnrepository.com/artifact/net.snowflake/spark-snowflake_2.12/2.8.4-spark_3.0
# Please look for another version based on the values you find above for your spark and scala version.

In [15]:
!wget -N https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.8.4-spark_3.0/spark-snowflake_2.12-2.8.4-spark_3.0.jar -P /home/spark/shared/user-libs/spark2/

--2021-04-02 17:05:36--  https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.8.4-spark_3.0/spark-snowflake_2.12-2.8.4-spark_3.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.8.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.8.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710798 (694K) [application/java-archive]
Server file no newer than local file ‘/home/spark/shared/user-libs/spark2/spark-snowflake_2.12-2.8.4-spark_3.0.jar’ -- not retrieving.



In [16]:
!wget -N https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.12.17/snowflake-jdbc-3.12.17.jar -P /home/spark/shared/user-libs/spark2/

--2021-04-02 17:05:38--  https://repo1.maven.org/maven2/net/snowflake/snowflake-jdbc/3.12.17/snowflake-jdbc-3.12.17.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.8.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.8.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30823792 (29M) [application/java-archive]
Server file no newer than local file ‘/home/spark/shared/user-libs/spark2/snowflake-jdbc-3.12.17.jar’ -- not retrieving.



In [1]:
#We need to restart the kernel so will have new spark context that can avail the downloaded jar file

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210402170558-0001
KERNEL_ID = 17f930b0-d8f5-4c7e-a9b6-4e5b52684d5d


In [2]:
#Add snowdlake jdbc and spark snowflake connector jars to spark context so that they are copied to executors.
sc.addPyFile('/home/spark/shared/user-libs/spark2/spark-snowflake_2.12-2.8.4-spark_3.0.jar')
sc.addPyFile('/home/spark/shared/user-libs/spark2/snowflake-jdbc-3.12.17.jar')

In [60]:
#Uncomment this cell and add your credentials to snowflake database
#snowflake_credentials = {
#    'username': 'chuck',
#    'password': """XXXXXXX""",
#    'database': 'DEMO_DB',
#    'port': '50000',
#    'account_name': 'xxxxx',
#    'warehouse': 'COMPUTE_WH',
#    'role': 'ACCOUNTADMIN',
#    'schema': 'PUBLIC'
#}

In [56]:
# Snowflake options for Spark connector
sf_opts_spark = {
    "sfURL"       : snowflake_credentials['account_name']+".snowflakecomputing.com",  
    "sfRole"      : snowflake_credentials['role'],
    "sfUser"      : snowflake_credentials['username'],
    "sfPassword"  : snowflake_credentials['password'],
    "sfDatabase"  : snowflake_credentials['database'],
    "sfSchema"    : snowflake_credentials['schema'],
    "sfWarehouse" : snowflake_credentials['warehouse'],
}

In [57]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
query = """
select * from PUBLIC.CHUCKTEST limit 100
"""
df = spark.read.format(SNOWFLAKE_SOURCE_NAME).options(**sf_opts_spark).option("query", query).load()

In [58]:
df.show()

+---+-------+----+--------------------+
| ID|   NAME|RANK|                DATE|
+---+-------+----+--------------------+
|  1|charles|10.5|2021-04-02 16:59:...|
|  2|    sam| 5.5|2021-04-02 16:59:...|
|  3|  chloe| 2.8|2021-04-02 16:59:...|
|  4|    Amy| 4.9|2021-04-02 16:59:...|
+---+-------+----+--------------------+

